In [98]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import adam


[nltk_data] Downloading package stopwords to /home/yurii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from IPython.display import HTML

In [22]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data -O blood_data.csv

--2019-04-19 14:26:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12843 (13K) [application/x-httpd-php]
Saving to: ‘blood_data.csv’

blood_data.csv      100%[===================>]  12,54K  --.-KB/s    in 0s      

2019-04-19 14:26:03 (46,3 MB/s) - ‘blood_data.csv’ saved [12843/12843]



###### basic usage of regex


In [3]:
words = ['very','nice','lecture','day','moon']
expression = '|'.join(words)
expression

'very|nice|lecture|day|moon'

In [4]:
re.findall(expression, 'i attended a very nice lecture last year', re.M)

['very', 'nice', 'lecture']

###### Preprocessing the text 

In [5]:
sentence = 'John has been selected for the trial phase this time. Congrats!!'
sentence=sentence.lower()
sentence=sentence.replace('!','').replace('.', '')
words= sentence.split(' ')

# Define positive & negative words
positive_words=['awesome','good', 'nice', 'super', 'fun', 'delightful','congrats']
negative_words=['awful','lame','horrible','bad']

set(words)-set(positive_words)

{'been',
 'for',
 'has',
 'john',
 'phase',
 'selected',
 'the',
 'this',
 'time',
 'trial'}

###### Remove stopwords

In [6]:
tokens = nltk.word_tokenize(sentence)

stop_words = set(stopwords.words('english'))
new_tokens = [w for w in tokens if not w in stop_words]

print(f'Sentence with stop-words: {tokens}')
print(f'Sentence without stop-words: {new_tokens}')

Sentence with stop-words: ['john', 'has', 'been', 'selected', 'for', 'the', 'trial', 'phase', 'this', 'time', 'congrats']
Sentence without stop-words: ['john', 'selected', 'trial', 'phase', 'time', 'congrats']


In [7]:
tokens

['john',
 'has',
 'been',
 'selected',
 'for',
 'the',
 'trial',
 'phase',
 'this',
 'time',
 'congrats']

In [8]:
len(stop_words), type(stop_words)

(179, set)

###### Vectorization

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

Count how many times a specific word occurs in a text

In [10]:
vectorizer = CountVectorizer()
vectorizer_fit = vectorizer.fit_transform(sentence.split()).toarray()
feature_names = vectorizer.get_feature_names()

# print()
# print(vectorizer_fit.toarray())
print(sentence)
l = [print(row, feature_names[i]) for i, row in enumerate(vectorizer_fit)]

print('\n')

sentence2 = 'She should have been provided with a safe environment to recall her traumatic experiences. Instead she was filmed by the officer in charge on his phone as she described the ordeal.'
vectorizer_fit = vectorizer.fit_transform(sentence2.split()).toarray()
feature_names = vectorizer.get_feature_names()

l = [print(vectorizer_fit[i], word) for i, word in enumerate(feature_names)]



john has been selected for the trial phase this time congrats
[0 0 0 0 1 0 0 0 0 0 0] been
[0 0 0 1 0 0 0 0 0 0 0] congrats
[1 0 0 0 0 0 0 0 0 0 0] for
[0 0 0 0 0 0 1 0 0 0 0] has
[0 0 1 0 0 0 0 0 0 0 0] john
[0 0 0 0 0 0 0 1 0 0 0] phase
[0 0 0 0 0 0 0 0 0 0 1] selected
[0 0 0 0 0 1 0 0 0 0 0] the
[0 0 0 0 0 0 0 0 1 0 0] this
[0 0 0 0 0 0 0 0 0 1 0] time
[0 1 0 0 0 0 0 0 0 0 0] trial


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0] as
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0] been
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] by
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] charge
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0] described
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1] environment
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] experiences
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0] filmed
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] have
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

###### TF-IDF  == terms frequency & inverse document frequency

$$
TF = \frac{\sum{specWords}}{\sum{Words}};
$$
$$
IDF = \log{\frac{\sum{docs}}{\sum{docsWithWord}}}
$$
Regarding to this:
$$
TF-IDF = TF * IDF == \frac{\sum{specWords}}{\sum{Words}} \cdot \log{\frac{\sum{docs}}{\sum{docsWithWord}}}
$$

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
text = 'A thorough list of publicly available NLP data sets has already been created by Nicolas Iderhoff. Beyond these, here are some projects I can recommend to any NLP novice wanting to get their hands dirty:'
text = text.split('.')
print(text)
vector = TfidfVectorizer()
X = vector.fit_transform(text)
X.todense()

['A thorough list of publicly available NLP data sets has already been created by Nicolas Iderhoff', ' Beyond these, here are some projects I can recommend to any NLP novice wanting to get their hands dirty:']


matrix([[0.26255634, 0.        , 0.        , 0.26255634, 0.26255634,
         0.        , 0.26255634, 0.        , 0.26255634, 0.26255634,
         0.        , 0.        , 0.        , 0.26255634, 0.        ,
         0.26255634, 0.26255634, 0.26255634, 0.186811  , 0.        ,
         0.26255634, 0.        , 0.26255634, 0.        , 0.26255634,
         0.        , 0.        , 0.        , 0.26255634, 0.        ,
         0.        ],
        [0.        , 0.22641916, 0.22641916, 0.        , 0.        ,
         0.22641916, 0.        , 0.22641916, 0.        , 0.        ,
         0.22641916, 0.22641916, 0.22641916, 0.        , 0.22641916,
         0.        , 0.        , 0.        , 0.1610991 , 0.22641916,
         0.        , 0.22641916, 0.        , 0.22641916, 0.        ,
         0.22641916, 0.22641916, 0.22641916, 0.        , 0.45283832,
         0.22641916]])

In [13]:
from textblob.classifiers import NaiveBayesClassifier
data = [
 ('I love my country.', 'positive'),
 ('This is an amazing place!', 'positive'),
 ('I do not like the smell of this place.', 'negative'),
 ('I do not like this restaurant', 'negative'),
 ('I am tired of hearing your nonsense.', 'negative'),
 ("I always aspire to be like him", 'positive'),
 ("It's a horrible performance.", "negative")
 ]
model = NaiveBayesClassifier(data)

In [14]:
model.classify('Dont watch it')

'positive'

### Keras example

In [24]:
df = pd.read_csv('blood_data.csv', delimiter=',')

In [26]:
df.sample(11)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
460,21,1,250,21,0
309,16,3,750,19,0
499,74,1,250,74,0
144,4,2,500,9,1
93,4,2,500,4,0
340,14,4,1000,30,0
533,2,4,1000,14,0
256,16,8,2000,28,0
17,2,15,3750,49,1
608,4,1,250,4,0


Split features & true output

In [33]:
y = df['whether he/she donated blood in March 2007']
X = df.drop(columns='whether he/she donated blood in March 2007')

In [37]:
X.sample(10)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
328,14,4,1000,28
450,23,3,750,33
167,2,1,250,2
377,14,1,250,14
120,2,3,750,16
19,2,3,750,4
273,4,5,1250,58
61,2,8,2000,35
576,3,4,1000,29
47,2,2,500,2


In [39]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 4 columns):
Recency (months)         748 non-null int64
Frequency (times)        748 non-null int64
Monetary (c.c. blood)    748 non-null int64
Time (months)            748 non-null int64
dtypes: int64(4)
memory usage: 23.5 KB


In [42]:
X.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
count,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086
std,8.095396,5.839307,1459.826781,24.376714
min,0.000000,1.000000,250.000000,2.000000
25%,2.750000,2.000000,500.000000,16.000000
50%,7.000000,4.000000,1000.000000,28.000000
75%,14.000000,7.000000,1750.000000,50.000000
max,74.000000,50.000000,12500.000000,98.000000


###### Design a multilayer perceptron with Keras 

In [43]:
mlp = Sequential()
mlp.add(Dense(8, input_dim=X.shape[1], init='uniform', activation='relu'))
mlp.add(Dense(6, init='uniform', activation='relu'))
# Last layer has sigmoid activation function to output values between 0 & 1
mlp.add(Dense(1, init='uniform', activation='sigmoid'))

/home/yurii/jupyter_envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, input_dim=4, activation="relu", kernel_initializer="uniform")`
  
/home/yurii/jupyter_envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yurii/jupyter_envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  """


Add optimizer

In [110]:
adam_opt = adam(lr=0.01)

Compile the model

In [111]:
mlp.compile(loss='binary_crossentropy', optimizer=adam_opt, metrics=['accuracy'])

In [112]:
mlp.layers

& Train!

In [113]:
mlp.fit(X, y, nb_epoch=200, batch_size=8, verbose=1)

/home/yurii/jupyter_envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/200
748/748 [==============================] - 0s 638us/step - loss: 0.5417 - acc: 0.7620
Epoch 2/200
748/748 [==============================] - 0s 241us/step - loss: 0.5040 - acc: 0.7620
Epoch 3/200
748/748 [==============================] - 0s 254us/step - loss: 0.5114 - acc: 0.7620
Epoch 4/200
748/748 [==============================] - 0s 247us/step - loss: 0.5133 - acc: 0.7620
Epoch 5/200
748/748 [==============================] - 0s 241us/step - loss: 0.5060 - acc: 0.7620
Epoch 6/200
748/748 [==============================] - 0s 232us/step - loss: 0.5067 - acc: 0.7620
Epoch 7/200
748/748 [==============================] - 0s 238us/step - loss: 0.5067 - acc: 0.7620
Epoch 8/200
748/748 [==============================] - 0s 254us/step - loss: 0.4920 - acc: 0.7620
Epoch 9/200
748/748 [==============================] - 0s 242us/step - loss: 0.5164 - acc: 0.7620
Epoch 10/200
748/748 [==============================] - 0s 244us/step - loss: 0.5056 - acc: 0.7620
Epoch 11/200
748/74

748/748 [==============================] - 0s 251us/step - loss: 0.5074 - acc: 0.7620
Epoch 84/200
748/748 [==============================] - 0s 226us/step - loss: 0.5074 - acc: 0.7620
Epoch 85/200
748/748 [==============================] - 0s 235us/step - loss: 0.5024 - acc: 0.7620
Epoch 86/200
748/748 [==============================] - 0s 249us/step - loss: 0.5041 - acc: 0.7620
Epoch 87/200
748/748 [==============================] - 0s 244us/step - loss: 0.4965 - acc: 0.7620
Epoch 88/200
748/748 [==============================] - 0s 236us/step - loss: 0.5069 - acc: 0.7620
Epoch 89/200
748/748 [==============================] - 0s 237us/step - loss: 0.4960 - acc: 0.7620
Epoch 90/200
748/748 [==============================] - 0s 230us/step - loss: 0.5190 - acc: 0.7620
Epoch 91/200
748/748 [==============================] - 0s 233us/step - loss: 0.5074 - acc: 0.7620
Epoch 92/200
748/748 [==============================] - 0s 234us/step - loss: 0.5009 - acc: 0.7620
Epoch 93/200
748/748 [=

748/748 [==============================] - 0s 242us/step - loss: 0.4960 - acc: 0.7620
Epoch 166/200
748/748 [==============================] - 0s 238us/step - loss: 0.4941 - acc: 0.7620
Epoch 167/200
748/748 [==============================] - 0s 228us/step - loss: 0.5217 - acc: 0.7620
Epoch 168/200
748/748 [==============================] - 0s 234us/step - loss: 0.5006 - acc: 0.7620
Epoch 169/200
748/748 [==============================] - 0s 240us/step - loss: 0.4941 - acc: 0.7620
Epoch 170/200
748/748 [==============================] - 0s 235us/step - loss: 0.5066 - acc: 0.7620
Epoch 171/200
748/748 [==============================] - 0s 231us/step - loss: 0.4933 - acc: 0.7620
Epoch 172/200
748/748 [==============================] - 0s 235us/step - loss: 0.5045 - acc: 0.7620
Epoch 173/200
748/748 [==============================] - 0s 238us/step - loss: 0.4946 - acc: 0.7620
Epoch 174/200
748/748 [==============================] - 0s 235us/step - loss: 0.5063 - acc: 0.7620
Epoch 175/200


In [49]:
mlp.evaluate(X,y)

748/748 [==============================] - 0s 86us/step


[0.4786351006278858, 0.7620320855614974]

In [125]:
from sklearn.linear_model import LogisticRegression

In [128]:
logreg = LogisticRegression()
clf = LogisticRegression(random_state=0).fit(X, y)

/home/yurii/jupyter_envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [135]:
(clf.predict(X) != y).sum() / X.shape[0]

0.22994652406417113